### Preparation

In [13]:
#!pip install numpy
#!pip install pandas
#!pip install requests
#!pip install tqdm
#!pip install sqlalchemy

### Packages

In [1]:
import numpy as np
import pandas as pd
import requests
import os
from tqdm import tqdm
from sqlalchemy import create_engine

### Clear documents

In [2]:
if os.path.exists("Journal.xlsx"):
    os.remove("Journal.xlsx")
    
if os.path.exists("Crypto.db"):
    os.remove("Crypto.db")

### Input

In [3]:
coins = ('BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'SOLUSDT', 'ADAUSDT', 'DOTUSDT', 'DOGEUSDT', 'AVAXUSDT', 'SHIBUSDT',
         'MATICUSDT', 'LTCUSDT', 'UNIUSDT', 'ALGOUSDT', 'TRXUSDT', 'LINKUSDT', 'MANAUSDT', 'ATOMUSDT', 'VETUSDT', 'FTMUSDT')

capital = 100 #$
UTC = 7
SMA = 28 #Simple Moving Average
timeframe = '4h'
#tradingfee = 0

### Download data

In [4]:
def download_data(symbol, timeframe):
    
    resp = requests.get('https://api.binance.us/api/v3/klines?symbol=' + symbol + '&interval=' + timeframe + '&limit=1000')
    frame = pd.DataFrame(resp.json())
    frame = frame.iloc[:,:5]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close']
    frame[['Open', 'High', 'Low', 'Close']] = frame[['Open', 'High', 'Low', 'Close']].astype(float)
    frame.Time = pd.to_datetime(frame.Time, unit='ms') + pd.Timedelta(hours=UTC)
    return frame

In [5]:
engine = create_engine('sqlite:///Crypto.db')

for coin in tqdm(coins):
    download_data(coin, timeframe).to_sql(coin, engine, index=False)

100%|██████████| 19/19 [00:19<00:00,  1.00s/it]


### Sort coin with high return compared to hold from backtesting

In [6]:
def technicals(df):
    df = df.copy()
    df['SMA'] = df.Close.rolling(SMA).mean()
    df.dropna(inplace=True)
    df['change'] = df.Close.pct_change()
        
    df['long strategy']  = np.where(df['Close'] > df['SMA'], 1, 0)
    df['Long positions'] = capital*(np.cumprod(df['long strategy'].shift(1)*df['change'] + 1))
        
    df['short strategy']  = np.where(df['Close'] > df['SMA'], 0, -1)
    df['Short positions'] = capital*(np.cumprod(df['short strategy'].shift(1)*df['change'] + 1))
        
    df['Return SMA ($)'] = df['Long positions']*df['Short positions']/capital
    df['Return hold ($)']  = capital*np.cumprod(df['change']+1)
    
    df.dropna(inplace=True)

    return df[['Return SMA ($)', 'Return hold ($)']]

In [7]:
profits = []
journal = pd.DataFrame()

for coin in coins:
    df = pd.read_sql(coin, engine).set_index('Time')
    ret = technicals(df)
    journal[coin] = ret.loc[:,'Return SMA ($)']
    profits.append(ret.iloc[-1,:])


frame = pd.DataFrame(profits, coins)
frame['edges ($)'] = frame['Return SMA ($)'] - frame['Return hold ($)']
frame = frame.sort_values('edges ($)', ascending=False)
frame

,Return SMA ($),Return hold ($),edges ($)
AVAXUSDT,186.968226,52.165276,134.802950
ETHUSDT,188.981994,65.990502,122.991492
SOLUSDT,136.070541,35.118966,100.951575
SHIBUSDT,169.916751,83.915441,86.001309
BTCUSDT,125.835090,53.482957,72.352134
ATOMUSDT,175.681564,113.092200,62.589364
DOGEUSDT,154.105637,105.057313,49.048324
BNBUSDT,138.617666,93.486451,45.131214
ADAUSDT,76.993891,52.467392,24.526499
FTMUSDT,73.223140,51.226388,21.996752


### Journal excel

In [8]:
journal.to_excel('Journal.xlsx', engine='xlsxwriter') 